In [ ]:
!pip install -q transformers datasets peft accelerate bitsandbytes

In [ ]:
# LORA Fine-Tuning Falcon-1B for Professional Email Generation

# ✅ STEP 1: Install Required Libraries
# (Uncomment the below lines when running in Colab)
# !pip install -q transformers datasets peft accelerate bitsandbytes

# ✅ STEP 2: Load Dataset
import json
from datasets import load_dataset

data_path = "/mnt/data/corporate_email_alpaca.jsonl"
dataset = load_dataset("json", data_files={"train": data_path})

# ✅ STEP 3: Load Tokenizer and Model in 8-bit
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "tiiuae/falcon-rw-1b"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token  # Falcon models often have no pad_token

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
    llm_int8_skip_modules=[]
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# ✅ STEP 4: Prepare LoRA
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# ✅ STEP 5: Preprocess Dataset
instruction_template = "Instruction: {}\nInput: {}\nOutput:"

def format_instruction(example):
    example["text"] = instruction_template.format(example["instruction"], example["input"]) + " " + example["output"]
    return example

dataset = dataset.map(format_instruction)

# ✅ Tokenize
def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize, remove_columns=dataset["train"].column_names)

# ✅ STEP 6: Training
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./falcon_lora_email_model",
    per_device_train_batch_size=1,
    num_train_epochs=3,
    logging_steps=5,
    save_total_limit=1,
    save_strategy="epoch",
    fp16=False,
    evaluation_strategy="no"
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

# ✅ STEP 7: Save LoRA Adapter
model.save_pretrained("./falcon_lora_email_model")
tokenizer.save_pretrained("./falcon_lora_email_model")

# ✅ STEP 8: Test Prompt
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

prompt = "Instruction: Write a professional corporate email\nInput: Requesting leave for medical reasons\nOutput:"
output = pipe(prompt, max_new_tokens=200, do_sample=True, top_k=50, temperature=0.7)
print(output[0]['generated_text'])
